# Workflow
In this example, we will use McStas 3 simulation file.

## Build Pipeline (Collect Parameters and Providers)
Import the providers from ``load_mcstas_nexus`` to use the ``McStas`` simulation data workflow. <br>
``MaximumProbability`` can be manually provided to derive more realistic number of events. <br>
It is because ``weights`` are given as probability, not number of events in a McStas file. <br>

In [ ]:
import sciline as sl

from ess.nmx.mcstas_loader import providers as loader_providers
from ess.nmx.mcstas_xml import read_mcstas_geometry_xml
from ess.nmx.data import small_mcstas_3_sample, small_mcstas_2_sample
from ess.nmx.types import *
from ess.nmx.reduction import NMXData, NMXReducedData, bin_time_of_arrival, TimeBinSteps


pl = sl.Pipeline((*loader_providers, read_mcstas_geometry_xml, bin_time_of_arrival))
# Replace with the path to your own file
pl[FilePath] = small_mcstas_3_sample()
pl[MaximumProbability] = 10000
pl[TimeBinSteps] = 50
# DetectorIndex selects what detector panels to include in the run
# in this case we select all three panels.
pl.set_param_table(sl.ParamTable(RunID, {DetectorIndex: range(3)}, index=range(3)))

To see what the pipeline can produce, display it:

In [ ]:
pl

## Build Workflow

In [ ]:
nmx_workflow = pl.get(NMXReducedData)
nmx_workflow.visualize()

## Compute Desired Types

In [ ]:
# Event data grouped by pixel id for each of the selected detectors
dg = nmx_workflow.compute(sl.Series[RunID, NMXData])
dg

In [ ]:
# Data from all selected detectors binned by panel, pixel and timeslice
binned_dg = nmx_workflow.compute(NMXReducedData)
binned_dg

## Export Results

``NMXReducedData`` object has a method to export the data into nexus or h5 file.

You can save the result as ``test.nxs`` for example.

```python
binned_dg.export_as_nexus('test.nxs')
```

In [ ]:
binned_dg.export_as_nexus('test.nxs')

## Instrument View

Pixel positions are not used for later steps,
but it is included in the coordinates for instrument view.

All pixel positions are relative to the sample position,
therefore the sample is at (0, 0, 0).

**It might be very slow or not work in the ``VS Code`` jupyter notebook editor.**

In [ ]:
import scippneutron as scn

da = dg[0]['weights']
da.coords['position'] = dg[0]['position']['panel', 0]
# Plot one out of 100 pixels to reduce size of docs output
view = scn.instrument_view(da['id', ::100].hist(), pixel_size=0.0075)
view.children[0].toolbar.cameraz()
view